## Домашняя работа 5

**Задание простого уровня** Мы говорили, что метрики качества нужны, чтобы сравнивать различные модели между собой. В задаче полиномиальной регрессии реализуйте код для выбора лучшей степени полиному:

* возьмите все степени от 1 до 10 по порядку, без пропусков.
* найдите степень полинома будет лучший r2-score
* напишите код, который выводит самую подходящую степень полинома и соответствующий ей скор

Эта процедура называется Grid Search и помогает найти лучшие параметры для модели.

Обучите лучшую модель и сделайте predict

In [11]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

data = pd.read_csv('../non_linear.csv', sep=',')
data.head()

,x_train,y_train
0,0.138368,0.838812
1,0.157237,0.889313
2,0.188684,1.430040
3,0.685553,1.717309
4,0.874237,2.032588


In [3]:
def generate_degrees(source_data: list, degree: int):
    """Функция, которая принимает на вход одномерный массив, а возвращает n-мерный
    
    Для каждой степени от 1 до  degree возводим x в эту степень
    """
    return np.array([
          source_data**n for n in range(1, degree + 1)  
    ]).T

In [37]:
def r2_score_polynomial(degree, data):
    """Генерим данные, тренируем модель
    
    дополнительно рисуем график
    """
    
    X = generate_degrees(data['x_train'], degree)

    model = LinearRegression().fit(X, data.y_train)
    y_pred = model.predict(X)

    error = r2_score(data['y_train'], y_pred)
    return error 

In [58]:
def find_best_polinom(data, max_n):
    errors = np.array([r2_score_polynomial(n, data) for n in range(1,max_n+1) ])    
    best_degree = np.argmax(errors)+1
    return best_degree, errors[best_degree-1] 

In [64]:
result = find_best_polinom(data, 10)
print('подходящую степень полинома %s, ошибка %s' %  result )

подходящую степень полинома 10, ошибка 0.9091133831293586


In [66]:
X = generate_degrees(data['x_train'], 10)

model = LinearRegression().fit(X, data.y_train)
y_pred = model.predict(X)
y_pred

array([ 0.88210097,  1.01897622,  1.21593663,  1.90777397,  1.89320068,
        1.96799159,  1.99510714,  2.00255713,  2.05113541,  2.05904861,
        2.07135515,  2.08488904,  2.0878935 ,  2.05782442,  1.96152895,
        1.70752908,  1.25295913,  1.19221964,  1.03691937,  1.00291056,
        0.88643136,  0.87018964,  0.48418417,  0.44549163,  0.29608611,
        0.26126515,  0.20252298,  0.19922475,  0.17116326,  0.15216131,
        0.15216131,  0.15070403,  0.14577292,  0.13117661,  0.0936225 ,
        0.08594859,  0.07275958,  0.06757109, -0.0088838 , -0.00456109,
        0.00499679,  0.0586599 ,  0.10034354,  0.22510037,  0.27123177,
        0.37545173,  0.56831688,  0.96442833,  0.93519255,  0.93519255])

**Задание среднего уровня** Напишите класс для обучения модели, который содержит:

* функцию `.fit(X, y)` , которая принимает на вход массив фичей `X`, массив таргетов `y` и обучает коэффициенты регрессии. Код для обучения взять из первого урока модуля *Постановка ML задачи линейной регрессии*
* функцию `.predict(X)`, которая по массиву фичей `X` возвращает массив предсказаний `y`

Нужно использовать код для аналитически вычисяемых коэффициентов. 

Это задание позволит понять, как работает линейная регрессия "внутри" библиотечной реализации.

In [131]:
import numpy as np
from numpy.linalg import inv

class CustomLinearReg:
    def __init__(self):
        self.w = []
    
    def fit(self, X, y):
        X_new = [1 for i in range(1+len(X)-1)]+X
        X_new = np.array(X_new).reshape(-1,2, order='F')
        Y_new = np.array(y).reshape(-1,1)
        X_T_X = (X_new.T).dot(X_new)
        X_T_X_inverted = inv(X_T_X)
        self.w = X_T_X_inverted.dot(X_new.T).dot(Y_new)
        return self
    
    def predict(self,X):
        X_new = [1 for i in range(1+len(X)-1)]+X
        X_new = np.array(X_new).reshape(-1,2, order='F')
        res = X_new.dot(self.w)
        print(res)

In [141]:
x_hw = [50, 60, 70, 100]
y_hw = [10, 15, 40, 45]

reg = CustomLinearReg().fit(x_hw, y_hw)

reg.predict(x_hw)

[[13.21428571]
 [20.35714286]
 [27.5       ]
 [48.92857143]]


**Задание высокого уровня**

1. разделите датасет с домами Бостона из Урока 2 (таргет и фичи) на две части: в одной части 80% датасета (назовём train) в другой 20% (назовём valid)
1. обучите модель только на train датасете
1. постройте предсказания valid датасете
1. Посчитайте  `r2 score` на валидационном сете

После этого примените к обеим датасетам z-преобразование и повторите шаги 2-4. Как изменилась метрика r2?

Это задание поможет понять, как валидировать линейную регрессию (и другие модели) на отложенной выборке.

In [204]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

boston_dataset = load_boston()

features_train = boston_dataset.data[:round(boston_dataset.data.shape[0]*0.8),:]
features_valid = boston_dataset.data[round(boston_dataset.data.shape[0]*0.8):,:]

y_train = boston_dataset.target[:round(boston_dataset.target.shape[0]*0.8)]
y_valid = boston_dataset.target[round(boston_dataset.target.shape[0]*0.8):]

reg = LinearRegression().fit(features_train, y_train)

y_pred = reg.predict(features_valid)

error_before = r2_score(y_valid, y_pred)


y_train = StandardScaler().fit_transform(y_train.reshape(-1,1)).reshape(-1) 
y_valid = StandardScaler().fit_transform(y_valid.reshape(-1,1)).reshape(-1)

reg = LinearRegression().fit(features_train, y_train)

y_pred = reg.predict(features_valid)

error_after = r2_score(y_valid, y_pred)
error_after

print('r2 score до z-преобразования %f, после %f' %(error_before, error_after))

r2 score до z-преобразования -0.252942, после 0.062840


После z-преобразования r2 score стал больше, значит ошибка уменьшилась